In [1]:
%pip install ultralytics

import base64

import cv2
import ultralytics
from openai import OpenAI
from ultralytics import YOLO
from ultralytics.utils.downloads import safe_download
from ultralytics.utils.plotting import Annotator, colors

ultralytics.checks()

Ultralytics 8.3.68  Python-3.9.19 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Setup complete  (28 CPUs, 31.7 GB RAM, 168.0/293.0 GB disk)


### Download the video

In [5]:
# # safe_download("https://github.com/ultralytics/assets/releases/download/v0.0.0/anpr-demo-video.mp4")

True

In [7]:
cap = cv2.VideoCapture("anpr-demo-video.mp4")
assert cap.isOpened(), "Error reading video file"

# Video writer
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
video_writer = cv2.VideoWriter("anpr-output.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Load the Ultralytics YOLO license plate detection model
model = YOLO("best.pt")

In [2]:
client = OpenAI(api_key="OPEN_AI_KEY")

# Define the text prompt
prompt = """
Can you extract the vehicle number plate text inside the image?
If you are not able to extract text, please respond with None.  # Fallback instruction
Only output text, please.  # Ensure no extra formatting
If any text character is not from the English language, replace it with a dot (.)  # Handle non-English characters, because OpenCV directly can't process these.
"""

def extract_data(base64_encoded_data):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_encoded_data}"}
                    }
                ]
            }
        ]
    )
    return response

In [ ]:
padding = 10
cap = cv2.VideoCapture("anpr-demo-video.mp4")
while cap.isOpened():
    sucess, img = cap.read()
    if not sucess:
        break

    results = model.predict(img)[0].boxes
    boxes = results.xyxy.cuda()
    clss = results.cls.cuda()

    ann = Annotator(img, line_width=3)
    for (box, cls) in zip(boxes, clss):
        h, w, _ = img.shape
        x1 = max((int(box[0]) - padding), 0)
        x2 = max((int(box[2]) + padding), w)
        y1 = max((int(box[1]) - padding), 0)
        y2 = max((int(box[3]) + padding), h)



0: 640x384 2 License_Plates, 82.5ms
Speed: 2.9ms preprocess, 82.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 License_Plate, 75.4ms
Speed: 0.0ms preprocess, 75.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 License_Plates, 65.8ms
Speed: 2.0ms preprocess, 65.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 License_Plates, 59.8ms
Speed: 5.0ms preprocess, 59.8ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 License_Plates, 55.9ms
Speed: 4.3ms preprocess, 55.9ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 License_Plates, 25.2ms
Speed: 3.0ms preprocess, 25.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 License_Plate, 27.9ms
Speed: 2.0ms preprocess, 27.9ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 License_Plates, 26.1ms
Speed: 4.3ms preprocess, 26